In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/tinyllama

Mounted at /content/drive
/content/drive/MyDrive/tinyllama


In [ ]:
!pip install accelerate peft bitsandbytes transformers trl -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requi

In [ ]:
import pandas as pd


data_type='Original'
#data_type='Synthetic'

# Load Data
df = pd.read_excel('LR_Dataset_Original_Sythetic_Final.xlsx')
#df = pd.read_excel('LR_Dataset_Original_Sythetic_Experiment_70.xlsx') # 20% of the training and validation sets


if data_type=='Original':
 df = df[df['Source'] == 'Original'] # Only original

df= df[['Sentence','Category' ,'Classification']]

df['Sentence'] = df['Sentence'].str.capitalize()
df=df.sample(frac=1).reset_index(drop=True)
df['Sentence'] = (df['Sentence']
                  .str.strip()
                  .str.replace(r'\n|\r', ' ', regex=True)
                  .str.replace(r'\s{2,}', ' ', regex=True))
df

,Sentence,Category,Classification
0,Several studies have demonstrated the positive...,RESULT,TRAINING
1,"However, social annotation designs in previous...",RESEARCH GAP,TRAINING
2,"Also, shin et al. (2022) extended the classifi...",DESCRIPTION,TEST
3,Additional participants might highlight some s...,LIMITATION,TRAINING
4,"In the special cases of kb, including multi-ar...",OVERALL,VALIDATION
...,...,...,...
695,Preliminary evidence suggests that burst scs m...,RESULT,TRAINING
696,"Further, the chosen baseline of randomly prese...",LIMITATION,TRAINING
697,"In terms of the local context, the authors bui...",DESCRIPTION,VALIDATION
698,Zhuo et al. (2022) present the attet to captur...,DESCRIPTION,TRAINING


In [ ]:
def create_instruction(row):
    sentence = row['Sentence']
    instruction = (
                "You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\n"
                "Categories:\n "
                "1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n "
                "2. RESEARCH GAP: Highlights the need for further research within the topic.\n"
                "3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n"
                "4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “found”, “demonstrated”, and “observed” or phrases like “the findings indicate”.\n"
                "5. LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a previous study.\n"
                "6. EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the words “we” or “our”.\n"
                "7. OTHER: Any text that does not fit the above categories.\n"
                "Procedure:\n"
                "1. Determine whether the subject of the sentence is a topic or a study.\n"
                "2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\n"
                "3. Provide the category number that best fits the sentence. Just provide the category number without any explanation.\n"

                f"Sentence: {sentence}.\n"
            )

    return instruction

In [ ]:
df['instruction'] = df.apply(create_instruction, axis=1)

df = df.rename(columns={'Category': 'response'})

df

,Sentence,response,Classification,instruction
0,Several studies have demonstrated the positive...,RESULT,TRAINING,You are a researcher that should assign a clas...
1,"However, social annotation designs in previous...",RESEARCH GAP,TRAINING,You are a researcher that should assign a clas...
2,"Also, shin et al. (2022) extended the classifi...",DESCRIPTION,TEST,You are a researcher that should assign a clas...
3,Additional participants might highlight some s...,LIMITATION,TRAINING,You are a researcher that should assign a clas...
4,"In the special cases of kb, including multi-ar...",OVERALL,VALIDATION,You are a researcher that should assign a clas...
...,...,...,...,...
695,Preliminary evidence suggests that burst scs m...,RESULT,TRAINING,You are a researcher that should assign a clas...
696,"Further, the chosen baseline of randomly prese...",LIMITATION,TRAINING,You are a researcher that should assign a clas...
697,"In terms of the local context, the authors bui...",DESCRIPTION,VALIDATION,You are a researcher that should assign a clas...
698,Zhuo et al. (2022) present the attet to captur...,DESCRIPTION,TRAINING,You are a researcher that should assign a clas...


In [ ]:
test_dataset =df[df['Classification'] == 'TEST']
test_dataset

,Sentence,response,Classification,instruction
2,"Also, shin et al. (2022) extended the classifi...",DESCRIPTION,TEST,You are a researcher that should assign a clas...
5,"Thus, users can omit their names or use other ...",LIMITATION,TEST,You are a researcher that should assign a clas...
6,Our work builds on and generalises the promine...,EXTENSION,TEST,You are a researcher that should assign a clas...
14,Leveraging whole expressions instead of only w...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
16,"Specifically, in the third study, they perform...",RESULT,TEST,You are a researcher that should assign a clas...
...,...,...,...,...
667,"In this work, we study developers’ refactoring...",EXTENSION,TEST,You are a researcher that should assign a clas...
670,It requires 1800 epochs to make resnet-110 con...,LIMITATION,TEST,You are a researcher that should assign a clas...
673,"In that sense, starke et al. showed that nudgi...",RESULT,TEST,You are a researcher that should assign a clas...
675,Another threat to validity is that some of the...,LIMITATION,TEST,You are a researcher that should assign a clas...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline

# Model Setup (Quantization and Tokenizer)

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, # This new
    device_map="auto",
    trust_remote_code=True,
)

# Tokenizer Initialization
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    add_bos_token=True,
    trust_remote_code=True,
)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "tinyllama_7B_Instruct-lro-finetune")

In [ ]:
def generate_test_prompt(example):
    """Format prompt for training."""
    text = f"<|im_start|>user\n{example['instruction']}<|im_end|>"
    return text

In [ ]:
test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


<ipython-input-10-ca85691956fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


In [ ]:
test_dataset

,Sentence,response,Classification,instruction,text
2,"Also, shin et al. (2022) extended the classifi...",DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
5,"Thus, users can omit their names or use other ...",LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
6,Our work builds on and generalises the promine...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
14,Leveraging whole expressions instead of only w...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
16,"Specifically, in the third study, they perform...",RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
...,...,...,...,...,...
667,"In this work, we study developers’ refactoring...",EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
670,It requires 1800 epochs to make resnet-110 con...,LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
673,"In that sense, starke et al. showed that nudgi...",RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
675,Another threat to validity is that some of the...,LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...


In [ ]:
def extract_text(text):
    # Define both markers to search for
    markers = ["<|im_end|>assistant", "assistant\n"]

    # Loop through markers and check if each is in the text
    for marker in markers:
        marker_position = text.find(marker)

        # If the marker is found, extract text after it
        if marker_position != -1:
            return text[marker_position + len(marker):].strip()  # Remove any leading/trailing whitespace

    # Return None if neither marker is found
    return text

In [ ]:
def clean_text(text):

    # Check for keywords and replace text accordingly
    if "OVERALL" in text:
        return "OVERALL"
    if "RESEARCH GAP" in text:
        return "RESEARCH GAP"
    if "DESCRIPTION" in text:
        return "DESCRIPTION"
    if "RESULT" in text:
        return "RESULT"
    if "LIMITATION" in text:
        return "LIMITATION"
    if "EXTENSION" in text:
        return "EXTENSION"
    elif "OTHER" in text:
        return "OTHER"

    # If neither keyword is found, return the original text
    return text

In [ ]:
prompt=test_dataset.iloc[0]['text']
prompt


'<|im_start|>user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “sh

In [ ]:
inputs = tokenizer(prompt,
                return_tensors="pt"
            ).to("cuda")

outputs = ft_model.generate(**inputs, max_new_tokens=25, use_cache=True)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

answer

'<|im_start|>user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “sh

In [ ]:
clean_text(extract_text(answer))

'DESCRIPTION'

In [ ]:
#pipe = pipeline(task="text-generation",
#                        model=ft_model,
#                        tokenizer=tokenizer,
#                        max_new_tokens = 100,
#                        temperature = 0.0,
#                       )
#result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
#answer = result[0]['generated_text']

In [ ]:
def get_classification_finetuning(data_point,model,tokenizer):
    """
    Gets the classification for a data point using the fine-tuned model.
    """
    inputs = tokenizer(data_point['text'],return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=25, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(clean_text(extract_text(answer)))
    data_point['Prediction_Finetune']=answer  # Assign the result to the data point
    data_point['Prediction_Finetune_Cleaned']=clean_text(extract_text(answer))

    return data_point
# Apply the get_classification function to the dataset using map
test_dataset = test_dataset.apply(lambda row: get_classification_finetuning(row, ft_model, tokenizer), axis=1)

DESCRIPTION
LIMITATION
EXTENSION
DESCRIPTION
RESULT
OTHER
EXTENSION
DESCRIPTION
OVERALL
EXTENSION
RESEARCH GAP
OTHER
RESULT
LIMITATION
OVERALL
LIMITATION
OTHER
RESULT
DESCRIPTION
RESEARCH GAP
OVERALL
DESCRIPTION
RESEARCH GAP
EXTENSION
OTHER
OVERALL
DESCRIPTION
EXTENSION
LIMITATION
RESEARCH GAP
EXTENSION
OVERALL
OVERALL
RESEARCH GAP
DESCRIPTION
RESULT
OTHER
OVERALL
RESULT
RESULT
OVERALL
OTHER
OVERALL
OTHER
LIMITATION
OVERALL
OVERALL
DESCRIPTION
OTHER
DESCRIPTION
LIMITATION
EXTENSION
OVERALL
RESULT
RESEARCH GAP
OTHER
RESULT
RESULT
OVERALL
OVERALL
OTHER
RESEARCH GAP
EXTENSION
DESCRIPTION
EXTENSION
DESCRIPTION
LIMITATION
RESEARCH GAP
RESULT
RESEARCH GAP
EXTENSION
OVERALL
OTHER
OTHER
EXTENSION
LIMITATION
OVERALL
OTHER
RESEARCH GAP
OVERALL
EXTENSION
LIMITATION
DESCRIPTION
DESCRIPTION
OTHER
LIMITATION
DESCRIPTION
RESULT
OVERALL
LIMITATION
RESULT
EXTENSION
RESULT
RESEARCH GAP
RESEARCH GAP
LIMITATION
DESCRIPTION
RESULT
RESEARCH GAP
RESULT
OTHER
LIMITATION
DESCRIPTION
OTHER
DESCRIPTION
OTHER
RES

In [ ]:
test_dataset


,Sentence,response,Classification,instruction,text,Prediction_Finetune,Prediction_Finetune_Cleaned
2,"Also, shin et al. (2022) extended the classifi...",DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,DESCRIPTION
5,"Thus, users can omit their names or use other ...",LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,LIMITATION
6,Our work builds on and generalises the promine...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,EXTENSION
14,Leveraging whole expressions instead of only w...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,DESCRIPTION
16,"Specifically, in the third study, they perform...",RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,RESULT
...,...,...,...,...,...,...,...
667,"In this work, we study developers’ refactoring...",EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,EXTENSION
670,It requires 1800 epochs to make resnet-110 con...,LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,LIMITATION
673,"In that sense, starke et al. showed that nudgi...",RESULT,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,RESULT
675,Another threat to validity is that some of the...,LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,LIMITATION


In [ ]:
#test_dataset.to_csv('TinyLlama_FT_Test_LoRA_Augmented.csv', index=False)
test_dataset.to_csv('TinyLlama_FT_Test_NEFT_Augmented.csv', index=False)
